YBIGTA 10기 박승리 노혜미

# CNN?

- 주로 이미지 인식에서 쓰이는 딥러닝 기술.
- 쉽게 말하자면, 하나의 이미지를 여러 층으로 쌓고 축소시키는 것을 통해 이미지 고유의 특성을 학습한다. 
- 새로운 이미지가 들어왔을 때, 학습한 것을 바탕으로 어떤 이미지인지 알려준다.

![cnn](http://fastml.com/images/cifar/lenet5.png)
출처: http://fastml.com/images/cifar/lenet5.png

# filter

![filter](http://i.imgur.com/OXwLhaf.gif)
출처: https://ratsgo.github.io/natural%20language%20processing/2017/03/19/CNN/

- 움직이면서 이미지의 정보를 얻어낸다.

# max pooling

![max pooling](https://qph.ec.quoracdn.net/main-qimg-8afedfb2f82f279781bfefa269bc6a90)
출처: https://qph.ec.quoracdn.net/main-qimg-8afedfb2f82f279781bfefa269bc6a90

- 필터를 움직이면서 필터 영역에서 최대인 값만을 뽑아낸다.

# ReLU

![relu](https://ml4a.github.io/images/figures/relu.png)
출처: https://ml4a.github.io/ml4a/neural_networks/

- sigmoid의 경우 어떤 x값을 넣더라도 0~1사이의 값으로 반환해주지만 vanishing gradient 문제를 가지고 있었다. 즉 모델이 train을 하기 위해서는 gradient가 필요한데 sigmoid때문에 gradient가 없어지는 문제가 발생했던 것이다.

- 하지만 ReLU는 음의 값을 넣어주면 0으로 만들어주고 양의 값을 넣어주면 그대로 통과시켜 이런 문제가 나타나지 않게 해준다.

- 예전에는 activate 함수로 sigmoid를 많이 썼지만 요즘은 ReLU를 많이 쓴다.

# drop out

![drop out](http://cs231n.github.io/assets/nn2/dropout.jpeg)
출처: http://cs231n.github.io/neural-networks-2/

- 일반적인 neural net은 neural끼리 모두 연결되어 있다.

- 하지만 이러한 neural net은 overfitting의 위험이 있다.

- 예를 들면, ybigta 사람들의 얼굴을 인식하는 cnn 모델을 만든다고 하자. train data로 사이언스 팀 사람들의 얼굴 사진을 넣었다. neural끼리 모두 연결되어있기 때문에, 눈 크기가 얼마라든지 눈썹 모양이라든지 train data의 아주 세세한 정보까지도 모델은 학습했다. 그래서 모델은 사이언스 팀에 overfitting 되어서 디자인팀과 엔지니어링팀의 얼굴을 인식하지 못하다든지 하는 상황이 발생하는 것이다.
 
- 이러한 overfitting을 방지하기 위해, train때 몇몇 뉴럴끼리의 연결을 끊어 놓은 뒤 test때 연결하는 방식은 **drop out**이라고 한다.

# MNIST를 이용한 실습

- minst를 이용해서 3개의 neural net을 쌓아보려고 한다.

소스코드 출처:

1.MNIST data를 가져온다.

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


2.dropout, x와 y값을 위한 변수들을 만들어준다.

In [3]:
# 스칼라이니 따로 shape를 주지 않는 것 같다.
keep_prob = tf.placeholder(tf.float32)

# None은 없다는 것이 아니라 n개의 의미를 가진다.
X = tf.placeholder(tf.float32, [None, 784])

# 28x28에 흑백이니 channel은 1이고(0~1사이의 값) 들어오는 이미지 갯수는 알 수 없으니 -1로 한다.
X_img = tf.reshape(X, [-1, 28, 28, 1])

# 0~9의 값이므로 10개
Y = tf.placeholder(tf.float32, [None, 10])

3.Layer 1

In [4]:
# 3x3이고 channel이 1인 filter를 32개 만들어준다. stddev는 표준편차.
# X_img에서 data의 갯수와 달리 filter의 갯수는 맨 뒤에 온다. 아마 그런 이유는 data처럼
# 단순히 나열되는 것이 아니라, 층으로 쌓이는 것이기 때문인 것 같다.
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))

# strides는 필터를 얼만큼 이동시킬 것인가를 설정해주는 것이다.
# pading을 SAME으로 하면 convoluntion 과정에서 이미지의 크기가 줄어드는 것을 방지해준다.
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')

L1 = tf.nn.relu(L1)

# pooling의 결과로 l1의 shpae는 (None, 14, 14, 32)이 된다. padding을 same으로 했음에도
# 사이즈가 줄어들었다. 궁금하다면 다음의 사이트를 참고하길 바란다.
# https://stackoverflow.com/questions/37674306/what-is-the-difference-between-same-and-valid-padding-in-tf-nn-max-pool-of-t
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

Tensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)

Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)

Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)

Tensor("dropout/mul:0", shape=(?, 14, 14, 32), dtype=float32)

4.Layer 2

In [5]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

# fully connected neural net과 이어주기 위해 입체적으로 되어있던 것을 일렬로 쭉 펴주는 
# 과정이다.
L2_flat = tf.reshape(L2, [-1, 7*7*64])

Tensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)

Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)

Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)

Tensor("dropout_1/mul:0", shape=(?, 7, 7, 64), dtype=float32)

Tensor("Reshape_1:0", shape=(?, 3136), dtype=float32)

5.Layer 3 (fully connected)

In [6]:
# W의 행은 들어오는 input크기이고 열은 out크기이다. (행렬곱의 결과를 생각해보면 된다.)
# xavier initailize는 난수를 난수의 최소 값의 제곱근으로 나누는 것이다. 예를 들면
# 2~4사이의 난수를 2^1/2로 나누는 것이다.
W3 = tf.get_variable("W3", shape=[3136, 10], initializer=tf.contrib.layers.xavier_initializer())

# random_normal은 정규분포에서 랜덤한 값을 얻겠다는 뜻이다.
b = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2_flat, W3) + b

6.cost/loss & optimizer 정의

In [7]:
# 단일 뉴럴네트워크에서 근거라는 게 여기서는 logits이다. 인풋 값이라고 생각하면 된다.
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

7.prediction/accuracy 정의

In [ ]:
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

8.training

In [9]:
# batch는 전체 data를 한번에 넣는 게 아니라 끊어서 넣는 것이다.
# epoch는 전체 data를 1번 돌았을 때의 단위를 말한다. 15 epoch는 전체 data를 15번 
# 돌겠다는 뜻이다.
# keep_prob는 텐서들 중 얼마를 활성화시킬 것인가를 나타낸다.
batch_size = 100

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(15):
        avg_cost=0
        total_batch = int(mnist.train.num_examples/batch_size)
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            cost_val, _ = sess.run([cost, optimizer], feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7})
            avg_cost += cost_val/total_batch
        
        print('Epoch:', '%04d' % (epoch + 1), 'average_cost =', '{:.9f}'.format(avg_cost))
    print('Accuracy:', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1}))

Epoch: 0001 average_cost = 0.455366684
Epoch: 0002 average_cost = 0.141064539
Epoch: 0003 average_cost = 0.106962732
Epoch: 0004 average_cost = 0.085217228
Epoch: 0005 average_cost = 0.073832755
Epoch: 0006 average_cost = 0.065674138
Epoch: 0007 average_cost = 0.057782949
Epoch: 0008 average_cost = 0.052816981
Epoch: 0009 average_cost = 0.049985569
Epoch: 0010 average_cost = 0.045589281
Epoch: 0011 average_cost = 0.042301787
Epoch: 0012 average_cost = 0.039655852
Epoch: 0013 average_cost = 0.037925539
Epoch: 0014 average_cost = 0.035527761
Epoch: 0015 average_cost = 0.034072179


9.testing

In [13]:
import random

In [37]:
# argmax는 벡터 값중 최대값의 index가 뭔지 알려준다. 그리고 grgmax에서 두 번째 인자는
# axis를 의미한다.

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # 랜덤하게 뽑아 test 해보자.
    r = random.randint(0, mnist.test.num_examples - 1)
    print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
    print("Prediction: ", sess.run(tf.argmax(hypothesis, 1), feed_dict={X: mnist.test.images[r:r + 1], keep_prob: 1}))

Label:  [7]
Prediction:  [7]
